In [11]:
import numpy as np
import pickle as pk
import pandas as pd
import os

from matplotlib import pyplot as plt

from asteroid_utils_pixell import  get_desig


In [192]:
def inv_var(data, variances):
    ave = 0
    var = 0
    for i in range(len(data)):
        ave += data[i]/variances[i]
        var += 1/variances[i]
    return ave/var, 1/var

In [351]:
flux_dir = '/scratch/r/rbond/jorlo/actxminorplanets/sigurd/fluxes/'

fluxes_090 = np.zeros((len(os.listdir(flux_dir)), 2))
fluxes_150 = np.zeros((len(os.listdir(flux_dir)), 3))

errs_090 = np.zeros((len(os.listdir(flux_dir)), 2))
errs_150 = np.zeros((len(os.listdir(flux_dir)), 3))

sn_090 = np.zeros(len(os.listdir(flux_dir)))
sn_150 = np.zeros(len(os.listdir(flux_dir)))

names = []


for i, ast_pk in enumerate(os.listdir(flux_dir)):
    name = ast_pk.split('_')[0]
    if 'normalized' in str(ast_pk): continue

    with open(flux_dir+ ast_pk, 'rb') as f:
        flux_dict = pk.load(f)
    
    names.append(name)
    
    flux_pa4_150, err_pa4_150 = flux_dict['night']['pa4']['150']['flux'], flux_dict['night']['pa4']['150']['var']
    flux_pa4_220, err_pa4_220 = flux_dict['night']['pa4']['220']['flux'], flux_dict['night']['pa4']['220']['var']

    flux_pa5_090, err_pa5_090 = flux_dict['night']['pa5']['090']['flux'], flux_dict['night']['pa5']['090']['var']
    flux_pa5_150, err_pa5_150 = flux_dict['night']['pa5']['150']['flux'], flux_dict['night']['pa5']['150']['var']

    flux_pa6_090, err_pa6_090 = flux_dict['night']['pa6']['090']['flux'], flux_dict['night']['pa6']['090']['var']
    flux_pa6_150, err_pa6_150 = flux_dict['night']['pa6']['150']['flux'], flux_dict['night']['pa6']['150']['var']
    
    if err_pa4_150 == 0:
        err_pa4_150 = 999999
    if err_pa5_150 == 0:
        err_pa5_150 = 999999
    if err_pa6_150 == 0:
        err_pa6_150 = 999999
    if err_pa4_220 == 0:
        err_pa4_220 = 999999
    if err_pa5_090 == 0:
        err_pa5_090 = 999999
    if err_pa6_090 == 0:
        err_pa6_090 = 999999
    
    fluxes_090[i] = np.array((flux_pa5_090, flux_pa6_090))
    errs_090[i] = np.array((err_pa5_090, err_pa6_090))
    
    fluxes_150[i] = np.array((flux_pa4_150, flux_pa5_150, flux_pa6_150))
    errs_150[i] = np.array((err_pa4_150, err_pa5_150, err_pa6_150))    
    
    tot_flux_090, tot_err_090 = inv_var(fluxes_090[i], errs_090[i]**2)
    sn_090[i] = tot_flux_090/tot_err_090
    
    tot_flux_150, tot_err_150 = inv_var(fluxes_150[i], errs_150[i]**2)
    sn_150[i] = tot_flux_150/tot_err_150


names = np.array(names)


In [352]:
sn_cut = 10

flags_090 = np.where((fluxes_090[...,0] != 0) & (fluxes_090[...,1] != 0) & (sn_090>=sn_cut))


In [353]:
fluxes_090 = fluxes_090[flags_090]
errs_090 = errs_090[flags_090]

names_090 = names[flags_090]

In [354]:
ratios_090 = (fluxes_090[...,0]-fluxes_090[...,1])/np.amax(fluxes_090, axis = 1)
ratio_errs_090 = np.sqrt((errs_090[...,0]/fluxes_090[...,0])**2 + (errs_090[...,1**2]/fluxes_090[...,1])**2)*ratios_090

In [355]:
inv_var(ratios_090, ratio_errs_090**2)

(-0.008864694808341678, 2.8280961926827373e-08)

In [356]:
np.mean(ratios_090)

0.012092098849041038

In [357]:
sn_cut = 0

flags_150 = np.where((fluxes_150[...,0] != 0) & (fluxes_150[...,1] != 0) & (fluxes_150[...,2] !=0) & (sn_150>=sn_cut))

fluxes_150 = fluxes_150[flags_150]
errs_150 = errs_150[flags_150]

names_150 = names[flags_150]

In [358]:
ratios_150 = (fluxes_150[...,0]-fluxes_150[...,1])/np.amax(fluxes_150, axis = 1)
ratio_errs_150 = np.sqrt((errs_150[...,0]/fluxes_150[...,0])**2 + (errs_150[...,1]/fluxes_150[...,1])**2)*ratios_150

In [359]:
inv_var(ratios_150, ratio_errs_150**2)

(-0.0036328798871196415, 8.038818433987138e-09)

In [360]:
np.mean(ratios_150)

-0.06534019839307077

In [361]:
ratios_150 = (fluxes_150[...,2]-fluxes_150[...,1])/np.amax(fluxes_150, axis = 1)
ratio_errs_150 = np.sqrt((errs_150[...,2]/fluxes_150[...,2])**2 + (errs_150[...,1]/fluxes_150[...,1])**2)*ratios_150

In [362]:
inv_var(ratios_150, ratio_errs_150**2)

(0.010618340551824996, 1.6260229638724773e-08)

In [363]:
np.mean(ratios_150)

-0.0663578443967689

In [364]:
ratios_150 = (fluxes_150[...,2]-fluxes_150[...,0])/np.amax(fluxes_150, axis = 1)
ratio_errs_150 = np.sqrt((errs_150[...,2]/fluxes_150[...,2])**2 + (errs_150[...,0]/fluxes_150[...,0])**2)*ratios_150

In [365]:
inv_var(ratios_150, ratio_errs_150**2)

(-0.009852086068043997, 4.369297156471608e-08)

In [366]:
np.mean(ratios_150)

-0.001017646003698107